In [129]:
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from bs4 import BeautifulSoup as Soup
from urllib.parse import urlparse, urljoin
import re

##dbs星展, sinopac永豐
bank_urls = {
            'dbs':'https://www.dbs.com.tw/personal-zh/cards/dbs-credit-cards/default.page', 
            'sinopac':'https://bank.sinopac.com/sinopacBT/personal/credit-card/introduction/list.html',
            'cathy': 'https://www.cathaybk.com.tw/cathaybk/personal/product/credit-card/cards/',
            'firstbank': 'https://card.firstbank.com.tw/sites/card/CreditCardList'
             }
credit_card_urls = {}
soup_documents = {k:None for k in list(bank_urls.keys())}

for bank, url in bank_urls.items():
    loader = RecursiveUrlLoader(url=url)
    docs = loader.load()
    if bank == 'dbs':
        links = list(set([a['href'] for a in Soup(docs[0].page_content).find_all('a', href=True) if (a['href'].startswith('/personal-zh/cards')) and (a['href'].endswith('hyperlink') or a['href'].endswith('cta'))]))
        credit_card_urls[bank] = [urljoin(url, i) for i in links]
    elif bank == 'sinopac':
        links = list(set([a['href'] for a in Soup(docs[0].page_content).find_all('a', href=True) if a['href'].startswith('./')]))
        credit_card_urls[bank] = [urljoin(url, i) for i in links]
    elif bank == 'firstbank':
        def firstbank_extractor(html:str):
            soup_obj = Soup(html, 'html.parser')
            credit_card_feature = [re.sub(r'\n+', '\n', d.text) for d in soup_obj.find_all('div', {"class":[["card-single-features"]]})]
            txt = ''
            for features in  credit_card_feature:
                split = features.split('\n', 1)
                txt += f"信用卡卡名：{split[0].strip()}\n信用卡特色: {split[1]}"
            return txt
        all_html = RecursiveUrlLoader(bank_urls['firstbank'], extractor=firstbank_extractor).load()
        soup_documents[bank] = all_html

        divs = [Soup(doc.page_content, 'html.parser').find_all('div', {"class":'card-single'}) for doc in docs][0]
        links = []
        for d in divs:
            link_tags = d.find_all('a')
            if len(link_tags) == 1:
                detail_url = link_tags[0]['href']
                links.append(detail_url)
            else:
                for tag in link_tags:
                    if tag.text == '詳細內容':
                        links.append(tag['href'])
                    else:
                        continue
        credit_card_urls[bank] = [urljoin(url, i) for i in links]
    elif bank == 'cathy':
        def parser(html):
            ## main page
            divs = Soup(html, 'html.parser').find_all('div', {'class':'cubre-m-compareCard -credit'})
            if divs:
                divs = [re.sub(r'\n+', '\n', i.text) for i in divs]
                txt = (' ').join(divs)
                txt = re.sub(r' +', ' ', txt)
                txt = [t.replace('\n立即申辦','', 1) for t in txt.split('詳細說明') if t.startswith('\n立即申辦')]
                txt = ('').join(txt).replace('\n \n', '\n')
            else:
                ## deeper page
                divs = Soup(html, 'html.parser').find_all('div', class_=["cubre-o-textContent", "cubre-m-colorBanner__title","cubre-m-iconEssay__title","cubre-m-horGraphic__title","cubre-m-remind__title","cubre-m-puzzle__title","cubre-a-kvTitle -card"])
                divs = [d.text for d in divs if '您將離開本行官網 前往外部網站' not in d.text]
                uni_divs = []
                for d in divs[:len(divs)-1]:
                    if d not in uni_divs:
                        uni_divs.append(d)
                txt = re.sub(r'\n+', '\n', ('\n').join(uni_divs))
                txt = re.sub(r' +', ' ', txt)
            return txt
            
        docs = RecursiveUrlLoader(url=url, extractor=parser).load()
        for d in docs:
            d.metadata['bank'] = '國泰銀行'
        soup_documents['cathy'] = docs
        

In [2]:
from langchain_community.document_loaders.url_selenium import SeleniumURLLoader
# documents = {}
# for bank, links in credit_card_urls.items():
#     documents[bank] = SeleniumURLLoader(urls=links).load()
        

In [ ]:
def sinopac_extractor(html: str) -> str:
    soup = Soup(html, "html.parser")
    divs_txt = [s.text for s in soup.find_all('div', {'class':'tab-box'})]
    divs_txt.insert(0, re.sub(r'\n\n+', '\n', soup.find('div', {'class':'info'}).text))
    div_set = []
    for txt in divs_txt:
        if txt not in div_set:
            div_set.append(txt)
    txt = re.sub(r'[\t\r\xa0]', '', ('\n').join(div_set))
    txt = re.sub(r'  +', ' ', txt)
    txt = re.sub(r'\n+', '\n', txt)
    return txt

def dbs_extractor(html:str)->str:
    soup = Soup(html, "html.parser")
    divs_txt = [s.text for s in soup.find_all('div', {'class':'flpweb-legacy'})]
    if divs_txt:
        div_set = []
        for txt in divs_txt:
            if txt not in div_set:
                div_set.append(txt)
        all_txt = ('\n').join(div_set)
    else:
        txt = soup.text
        txt = [t.strip() for t in txt.split('\n') if( t != 'more..') and (len(txt.strip()) != 1)]
        all_txt = ('\n').join(txt)

    all_txt = all_txt.replace('個人網路銀行\nCard+ 信用卡數位服務\n企業網路銀行', '')
    all_txt = re.sub(r'[\t\r\xa0]', '', all_txt)
    all_txt = re.sub(r'\xa0', '', all_txt)
    all_txt = re.sub(r'  +', ' ', all_txt)
    all_txt = re.sub(r'\n+', '\n', all_txt)
    all_txt = all_txt.split('信用卡刷卡優惠更多星展信用卡刷卡優惠信息，詳情請見刷卡優惠說明。')[0]

    if '假字假字假字假字假字假字假字假字假字假字' in all_txt:
        all_txt = re.sub('假字假字假字+', '', all_txt)

    pattern = r"網銀登入\n[\s\S]*?選擇您的網站 ▼\n個人金融\n個人金融\n財富管理\n星展豐盛理財\n星展豐盛私人客戶\n企業金融\n中小企業銀行\n企業及機構銀行\n星展集團\n關於我們"
    # Replace the matched portion with an empty string
    all_txt = re.sub(pattern, '', all_txt)
    pattern = r'Loading...\n尊享至上\n[\s\S]*?\n權益手冊\n更多獨享優惠'
    all_txt = re.sub(pattern, '', all_txt)
    pattern = r'新戶好禮\n[\s\S]*?\n更多星展信用卡'
    all_txt = re.sub(pattern, '', all_txt)
    all_txt = all_txt.split('謹慎理財 信用至上')[0]
    return all_txt

def firstbank_extractor(html:str)->str:
    divs = Soup(html, 'html.parser').find_all('div', {"class":['carousel-item', 'content-body']})
    return ('').join([re.sub(r'\n+', '\n', d.text) for d in divs])

for bank, links in credit_card_urls.items():
    docs = []
    if bank == 'sinopac':
        for url in links:
            doc = RecursiveUrlLoader(url=url, extractor=sinopac_extractor).load()
            doc[0].metadata['bank'] = '永豐銀行'
            docs.extend(doc)
    elif bank == 'dbs':
        for url in links:
            doc = RecursiveUrlLoader(url=url, extractor=dbs_extractor).load()
            if doc:
                doc[0].metadata['bank'] = '星展銀行'
                docs.extend(doc)
    elif bank == 'firstbank':
        for url in links:
            doc = RecursiveUrlLoader(url=url, extractor=firstbank_extractor).load()
            doc[0].metadata['bank'] = '第一銀行'
            docs.extend(doc)
    if soup_documents[bank]:
        soup_documents[bank].extend(docs)
    else:
        soup_documents[bank] = docs

In [131]:
soup_documents['dbs'][0].metadata['description'], soup_documents['dbs'][0].page_content

('海外消費2.52%／國內消費1.2%現金回饋，不只生活更要炫活',
 '\n星展炫晶御璽卡\n海外消費2.52%／國內消費1.2%現金回饋，不只生活更要炫活(目前不提供新卡申請)\n產品特色海外消費享2.52%現金回饋，國內消費享1.2%現金回饋當月消費達NT$2萬，次月享「台灣聯通停車場」每日一次最長2小時免費停車免責申明 × 提醒：您即將離開星展銀行（台灣）網站：所有其他業者經營之網站均由業者自行負責（包括客戶隱私權保護及客戶資訊安全事項），不屬於星展銀行（台灣）控制或負責範疇。若您同意繼續前往該網站請選擇「繼續」。不同意請選擇「結束」。謝謝！繼續 結束\n消費享最高2.52%現金回饋刷星展炫晶御璽卡，海外消費享2.52%現金回饋，國內消費享1.2%現金回饋，現金回饋自動折抵帳單消費金額，於持卡期間均有效。\n 星展炫晶御璽卡現金回饋注意事項星展炫晶御璽卡現金回饋活動僅適用於星展炫晶御璽卡正附卡持卡人。本活動合格消費僅限於一般消費而不包含下列列示項目：【信用卡年費、全聯福利中心之儲值／各項消費、歐洲經濟區實體商店通路之刷卡消費、循環信用利息費用、逾期手續費、預借現金金額（ex：分期預借現金專案之每月應攤還本金、ATM 提領現金、賭場兌換籌碼等）、分期付款之每月應攤還本金及其手續費及利息（ex：特約商店分期付款、分期輕鬆購、帳單輕鬆分、來電輕鬆分等分期付款專案）、境外投資交易平台（包含但不限於eToro）、餘額代償、各項稅款（ex：個人綜合所得稅、查核定稅、房屋稅、地價稅、營業稅、牌照稅等）、各項政府機關規費及罰款、中華電信各項電信資費、水／電／瓦斯（天然氣）等公用事業費用、學雜費、醫指付/電子化繳費稅處理平台/「聯合信用卡處理中心」公務機關信用卡繳費平台（適用之事業機構及醫療院所請依各繳費平台網頁公告為準）、e政府服務平台繳款（包括但不限於學雜費、政府機關規費等）、其他一般信用卡相關各項手續費用、提供之信用卡小額支付特約商店之消費（ex：便利商店、連鎖速食店等） 等。】新增國內消費（見 3.1.）享基本現金回饋比率為 1.2%，新增海外消費（見3.2.）除享基本現金回饋比率1.2%外，更加碼現金回饋比率至2.52%。3.1. 國內消費係指交易商店之國別為台灣地區且交易幣別新台幣之一般交易。3.2. 海外消費係指交易商店之國別非台灣地區或非以新臺幣為交易幣別並

In [25]:
for k,v in soup_documents.items():
    print(f"Bank {k}: {len(v)}")

Bank dbs: 34
Bank sinopac: 44
Bank cathy: 31
Bank firstbank: 37


### Summarize the documents

In [11]:
from langchain.chains.summarize import load_summarize_chain
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import ChatOllama

llm = ChatOllama(model='qwen2:7b-instruct', temperature=0)
prompt_template = """以下文字為星展銀行信用卡的資訊，請幫我撰寫簡潔的摘要，能夠清楚明瞭是什麼信用卡、優惠有什麼、注意事項及活動時間等重要資訊。請勿解釋或是添加前言，直接回覆答案即可。
輸出格式請以純文字輸出，勿以markdown方式呈現。
信用卡所有資訊:
{text}
答案:
  """

prompt_template = """將以下內容簡潔地總結為繁體中文:
{text}
簡潔的總結:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "您的工作是製作最終的摘要\n"
    "我們已經提供了一個現有的摘要，直到某個點為止: {existing_answer}\n"
    "我們有機會根據以下更多的背景信息"
    "如果需要）來完善現有的摘要\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "根據新的訊息，請將原始摘要進行修改（如果需要的話）。"
    "如果訊息不有幫助，請返回原始摘要。"
)
refine_prompt = PromptTemplate.from_template(refine_template)

dbs_prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

stuff_chain = load_summarize_chain(llm, chain_type="refine",
                                   question_prompt=prompt,
                                    refine_prompt=refine_prompt,
                                    # return_intermediate_steps=True,
                                    input_key="input_documents",
                                    output_key="output_text",)

try:
    print(stuff_chain.run([soup_documents['dbs'][3]]))
except Exception as e:
    print(
        "The code failed since it won't be able to run inference on such a huge context and throws this exception: ",
        e,
    )

信用卡产品介绍:

1. **万事达卡商务礼遇星展商务eco卡**:
   - 针对商界需求设计,提供商务优惠和支付便利。
   - 支持货款垫付、广告费用等多用途消费。
   - 享有网购保险服务,保障网购安全。

2. **万事达卡商务钛金卡**:
   - 商务等级信用卡,提供一系列商务礼遇和服务。
   - 持卡人可享受特定的网上购物保险权益。

3. **申办资格**:
   - 年龄18至75岁,年收入需达到新台币22万元以上。

4. **年费办法**:
   - 正卡年费为新台币3000元。
   - 持卡第二年起,满足特定条件可免缴当年年费。

5. **环保材质**:
   - 使用PLA聚乳酸材料,比传统塑料更天然、耐用且对环境影响较小。

6. **信用卡刷卡优惠**:
   - 详情请查询刷卡优惠说明页面。

7. **循环信用利率**:
   - 区间为5.99%至14.99%,基准日为民国113年6月28日。

8. **存款保险**:
   - 本机构已参加存款保险,提供一定的资金安全保障。


In [ ]:
soup_documents['dbs'][3].page_content

In [132]:
import mlflow
from langchain.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_qdrant import QdrantVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

splitter = RecursiveCharacterTextSplitter(chunk_size=400,
                                        length_function=len,
                                        is_separator_regex=True,
                                        chunk_overlap=50,
                                        separators=['。'])
banks = list(bank_urls.keys())
chunks = splitter.split_documents(soup_documents[banks[0]])
for b in banks[1:]:
    chunks.extend(splitter.split_documents(soup_documents[b]))

embedding = HuggingFaceEmbeddings(model_name='BAAI/bge-base-zh-v1.5', model_kwargs={'device':'mps'})
collection_name = 'credit_card'
qdrant_url = 'http://localhost:6333'
# vec_store = QdrantVectorStore.from_existing_collection(
#                               collection_name=collection_name,
#                               embedding = embedding,
#                               url = qdrant_url)

vec_store = QdrantVectorStore.from_documents(chunks,
                              collection_name=collection_name,
                              embedding = embedding,
                              force_recreate=True,
                              url = qdrant_url)



## Graph RAG

In [140]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm = ChatOllama(model='jack/llama3-8b-chinese')
llm_transformer = LLMGraphTransformer(llm=llm)

all_docs = soup_documents[list(soup_documents.keys())[0]]
for k in list(soup_documents.keys())[1:]:
    all_docs.extend(soup_documents[k])
# print(len(all_docs))

graph_documents = llm_transformer.convert_to_graph_documents(all_docs[:20])
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[]
Relationships:[]


In [13]:
from langchain.retrievers.multi_query import MultiQueryRetriever
retriever = vec_store.as_retriever(search_kwargs={'k':10, 'score_threshold':0.5})
    
llm = ChatOllama(model='qwen2:7b-instruct', temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=llm
)


In [19]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder 
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever
from langchain_core.runnables import RunnablePassthrough
import opencc

from opik.integrations.langchain import OpikTracer
import os
import opik
# opik.configure(use_local=True)

# Create the Opik tracer
# opik_tracer = OpikTracer(tags=["langchain", "ollama"])
# os.environ["OPIK_PROJECT_NAME"] = "credit-card-helper"
converter = opencc.OpenCC('s2t.json')
reranker = HuggingFaceCrossEncoder(model_name='BAAI/bge-reranker-base')
compressor = CrossEncoderReranker(model=reranker, top_n=10)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever_from_llm)

model = ChatOllama(model='qwen2:7b-instruct', temperature=0)
# .with_config({"callbacks": [opik_tracer]})

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI question-answering assistant. Your task is to answer the question based on the provided documents. The documents are part of the text from the description of the credit cards.
    The documents are not all relevant to the question. Please filter and reply with an answer. No pre-amble or explanation. Please output text only, not in markdown format.
    
    Documents: 
    {context}

    Question:
    {question}

    Answer:""",
)

def format_docs(docs):
    return "\n\n".join([f"DOCUMENT {index}\nTitle: {d.metadata['title']}\n{d.page_content}" for index, d in enumerate(docs)])


chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | QUERY_PROMPT
    | model
    | StrOutputParser()
)

question = "國泰銀行有什麼信用卡？"
response = chain.invoke(question)
# compression_retriever.invoke(question)
print(converter.convert(response))

國泰銀行有多款信用卡供選擇：

1. **世界卡**: 提供特定回饋優惠，但不包含某些費用和交易類型。
2. **聯合報優秀卡**: 已停發，提供年費、掛失費等各項費用及產品權益的資訊可參閱各產品網頁。
3. **NFC行動信用卡**: 已停發，不再受理申請與補發業務。

這些是國泰銀行提供的幾款信用卡類型。每張卡都有其特定的優點和適用條件，請根據自己的需求選擇適合的信用卡。


In [8]:
from opik.evaluation.metrics import Hallucination
from opik.evaluation.models import LiteLLMChatModel

model = LiteLLMChatModel(
    name="ollama/llama3.2",
    base_url="http://localhost:11434"
)

hallucination_metric = Hallucination(
    model=model
)

In [ ]:
hallucination_metric.score(
    input=question,
    output=response,
)

In [1]:
from litellm import completion

response = completion(
    model="ollama/llama3.2", 
    messages=[{ "content": "respond in 20 words. who are you?","role": "user"}], 
    api_base="http://localhost:11434"
)
print(response['choices'][0].message.content)


ModelResponse(id='chatcmpl-d7c53168-a465-429f-bfbb-6b957da8a265', created=1735090100, model='ollama/llama3.2', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content="I'm an artificial intelligence designed to provide helpful and informative responses, assisting users with various inquiries and tasks efficiently.", role='assistant', tool_calls=None, function_call=None))], usage=Usage(completion_tokens=23, prompt_tokens=38, total_tokens=61, completion_tokens_details=None, prompt_tokens_details=None))
